<a href="https://colab.research.google.com/github/semenovi/cuda-practice/blob/main/task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---

CUDA

removing cuda-related packages

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update


install cuda

In [ ]:
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0

check the installation

In [ ]:
!nvcc --version

install nvcc to cuda programming in c++

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

load this

In [ ]:
%load_ext nvcc_plugin

**DA PROGRAM**

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
int main() {
    printf("Hello world");
    return 0;
}



---

OPENMP

da file

In [ ]:
%%cuda --name omp_cuda.cu
#include <stdio.h>
#include <omp.h>
int main() {
    printf("Hello world");
    return 0;
}

da compile

In [ ]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/omp_cuda /content/src/omp_cuda.cu
!/content/src/omp_cuda


example 1

In [12]:
%%cuda --name example_1.cu
#include<omp.h>
#include<stdio.h>
main ()
{
  int size, rank;
 
  /* Создание множества параллельных процессов и в каждом из них задаются
  * свои приватные переменные size и rank */
 
  #pragma omp parallel private(size, rank)
  {
    /* Каждый процесс находит свой порядковый номер и выводит его на экран */
   
    rank = omp_get_thread_num();
    printf("Hello World from thread = %d\n", rank);
   
    /* Главный процесс - master выводит на экран количество процессов */
    if (rank == 0)
    {
     size = omp_get_num_threads();
     printf("Number of threads = %d\n", size);
    }
  }
 
  /* Завершение параллельной части */
 
}

'File written in /content/src/example_1.cu'

In [13]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/example_1 /content/src/example_1.cu
!/content/src/example_1

/content/src/example_1.cu(3): warning: return type "int" omitted in declaration of function "main"

/content/src/example_1.cu(3): warning: return type "int" omitted in declaration of function "main"

Hello World from thread = 1
Hello World from thread = 0
Number of threads = 2


In [15]:
%%cuda --name example_2.cu
#include <omp.h>
#include<stdio.h>
main ()
{
  int i, n;
  float a[100], b[100], sum;
  /* Инициализация элементов векторов */
  n = 100;
  for (i=0; i < n; i++)
    a[i] = b[i] = i * 1.0;
  sum = 0.0;
  /* Создание множества параллельных процессов и распараллеливание
  * цикла по виткам. При выходе из цикла все значения переменной sum
  * суммируются по всем процессам. */
  #pragma omp parallel for reduction(+:sum)
    for (i=0; i < n; i++)
      sum = sum + (a[i] * b[i]);
  /* Главный процесс выводит на экран значение sum */
  printf(" Sum = %f\n",sum);
}

'File written in /content/src/example_2.cu'

In [17]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/example_2 /content/src/example_2.cu
!/content/src/example_2

/content/src/example_2.cu(3): warning: return type "int" omitted in declaration of function "main"

/content/src/example_2.cu(3): warning: return type "int" omitted in declaration of function "main"

 Sum = 328350.000000


In [18]:
%%cuda --name example_3.cu
#include <omp.h>
#include<stdio.h>
#define VECLEN 100
float a[VECLEN], b[VECLEN], sum;
/* Подпрограмма, в которой суммируются элементы векторов */
float dotprod()
{
  int i,rank;
  rank = omp_get_thread_num();
  #pragma omp for reduction(+:sum)
  for (i=0; i < VECLEN; i++)
  {
    sum = sum + (a[i]*b[i]);
    printf(" rank = %d i=%d\n",rank,i);
  }
  return(sum);
}
main()
{
  int i;
  /* Инициализация элементов векторов */
  for (i=0; i < VECLEN; i++)
    a[i] = b[i] = 1.0 * i;
  sum = 0.0;
  /* Создание множества параллельных процессов */
  #pragma omp parallel
    sum = dotprod();
  printf("Sum = %f\n",sum);
}

'File written in /content/src/example_3.cu'

In [ ]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/example_3 /content/src/example_3.cu
!/content/src/example_3

In [20]:
%%cuda --name example_4.cu
#include <omp.h>
#include<stdio.h>
#define N 50
main()
{
  int i, size, rank;
  float a[N], b[N], c[N];
  /* Инициализация элементов векторов */
  for (i=0; i < N; i++)
    a[i] = b[i] = i * 1.0;
  /* Создание множества параллельных процессов */
  #pragma omp parallel shared(a,b,c) private(i,rank,size)
  {
    /* Каждый процесс находит свой порядковый номер и выводит его на экран */
    rank = omp_get_thread_num();
    printf("Thread %d starting...\n",rank);
    /* Директива задания секций */
    #pragma omp sections nowait
    {
      /* Секция 0*/
      #pragma omp section
      for (i=0; i < N/2; i++)
      {
        c[i] = a[i] + b[i];
        printf("rank = %d i= %d c[i]= %f\n", rank,i,c[i]);
      }
      /* Секция 1*/
      #pragma omp section
      for (i=N/2; i < N; i++)
      {
        c[i] = a[i] + b[i];
        printf("rank = %d i= %d c[i]= %f\n",
        rank,i,c[i]);
      }
    } /* Завершение блока секций */
    if (rank == 0)
    {
      size = omp_get_num_threads();
      printf("Number of threads = %d\n", size);
    }
  } /* Завершение параллельной части */
}

'File written in /content/src/example_4.cu'

In [ ]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/example_4 /content/src/example_4.cu
!/content/src/example_4

In [24]:
%%cuda --name example_5.cu
#include <omp.h>
#include<stdio.h>
#define M 10
main()
{
  float A[M][M], b[M], c[M];
  int i, j, rank;
  int total;
  /* Инициализация данных */
  for (i=0; i < M; i++)
  {
    for (j=0; j < M; j++)
    A[i][j] = (j+1) * 1.0;
    b[i] = 1.0 * (i+1);
    c[i] = 0.0;
  }
  printf("\Вывод значений матрицы A и вектора b на экран:\n");
  for (i=0; i < M; i++)
  {
    printf(" A[%d]= ",i);
    for (j=0; j < M; j++)
      printf("%.1f ",A[i][j]);
    printf(" b[%d]= %.1f\n",i,b[i]);
  }
  /* Создание множества параллельных процессов и в каждом из них задаются
  * свои приватные переменные rank и i*/
  #pragma omp parallel shared(A,b,c,total) private(rank,i)
  {
    rank = omp_get_thread_num();
    /* Директива распараллеливания цикла по виткам */
    #pragma omp for private(j)
      for (i=0; i < M; i++)
      {
        for (j=0; j < M; j++)
          c[i] += (A[i][j] * b[j]);
          /* Каждый процесс выводит свой порядковый номер, значение витка цикла и
          * значение результирующего вектора на каждом витке цикла и внутри
          * критической секции */
        #pragma omp critical
        {
          printf(" rank= %d i= %d c[%d]=%.2f\n", rank,i,c[i]);
        }
       }
   /* Конец параллельного цикла */
  }
 /* Завершение параллельной конструкции */
}

'File written in /content/src/example_5.cu'

In [ ]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/example_5 /content/src/example_5.cu
!/content/src/example_5

In [37]:
%%cuda --name 5.cu
#include <omp.h>
#include<stdio.h>
#define M 10
main()
{
  float A[M][M], B[M][M], C[M][M];
  int i, j, k, rank;
  int total;
  /* Инициализация данных */
  for (i=0; i < M; i++)
  {
    for (j=0; j < M; j++)
    {
      A[i][j] = (j+1) * 1.0;
      B[i][j] = (j+1) * 2.0;
      C[i][j] = 0.0;
    }
  }
  printf("\Вывод значений матрицы A:\n");
  for (i=0; i < M; i++)
  {
    printf(" A[%d]= ",i);
    for (j=0; j < M; j++)
      printf("%.1f ",A[i][j]);
    printf("\n");
  }
  printf("\Вывод значений матрицы B:\n");
  for (i=0; i < M; i++)
  {
    printf(" B[%d]= ",i);
    for (j=0; j < M; j++)
      printf("%.1f ",B[i][j]);
    printf("\n");
  }
  /* Создание множества параллельных процессов и в каждом из них задаются
  * свои приватные переменные rank и i*/
  #pragma omp parallel shared(A,B,C,total) private(rank,i,j)
  {
    rank = omp_get_thread_num();
    /* Директива распараллеливания цикла по виткам */
    #pragma omp for private(k)
      for (i=0; i < M; i++)
      {
        for (j=0; j < M; j++)
        {
            for (k=0; k < M; k++)
            {
              C[i][j] += A[i][k] * B[k][j];
            }
        /* Каждый процесс выводит свой порядковый номер, значение витка цикла и
        * значение результирующего вектора на каждом витке цикла и внутри
        * критической секции */
        #pragma omp critical
        {
          printf(" rank= %d i= %d j= %d C[%d,%d]=%.2f\n", rank,i,j,i,j,C[i,j]);
        }
        }
       }
   /* Конец параллельного цикла */
  }
 /* Завершение параллельной конструкции */
}

'File written in /content/src/5.cu'

In [38]:
!nvcc -Xcompiler="-fopenmp" -arch=sm_75 -o /content/src/5 /content/src/5.cu
!/content/src/5

/content/src/5.cu(4): warning: return type "int" omitted in declaration of function "main"

/content/src/5.cu(19): warning: unrecognized character escape sequence

/content/src/5.cu(27): warning: unrecognized character escape sequence

/content/src/5.cu(55): warning: expression has no effect

/content/src/5.cu(8): warning: variable "total" was declared but never referenced

/content/src/5.cu(4): warning: return type "int" omitted in declaration of function "main"

/content/src/5.cu(19): warning: unrecognized character escape sequence

/content/src/5.cu(27): warning: unrecognized character escape sequence

/content/src/5.cu(55): warning: expression has no effect

/content/src/5.cu: In function ‘int main()’:
/content/src/5.cu:55:74: warning: format ‘%f’ expects argument of type ‘double’, but argument 7 has type ‘float*’ [-Wformat=]
           printf(" rank= %d i= %d j= %d C[%d,%d]=%.2f\n", rank,i,j,i,j,C[i,j]);
                                                                          ^
В